In [2]:
import requests
import time
from multiprocessing import cpu_count
from multiprocessing.pool import ThreadPool
import pandas as pd
import os
import urllib.request

In [3]:
df = pd.read_csv("./csv-file/all_genre.csv")

In [4]:
df.head()

,id,preview_url,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,analysis_url,duration_ms
0,1EzrEOXmMH3G43AXT1y7pA,https://p.scdn.co/mp3-preview/28154061cbfda9d5...,acoustic,0.703,0.444,11,-9.331,1,0.0417,0.55900,0.000000,0.0973,0.712,150.960,https://api.spotify.com/v1/audio-analysis/1Ezr...,242947
1,4E6cwWJWZw2zWf7VFbH7wf,https://p.scdn.co/mp3-preview/7ed132f9327678a8...,acoustic,0.583,0.786,2,-3.142,0,0.0301,0.02080,0.000000,0.1880,0.573,123.055,https://api.spotify.com/v1/audio-analysis/4E6c...,258827
2,53QF56cjZA9RTuuMZDrSA6,https://p.scdn.co/mp3-preview/bf486b927761bf99...,acoustic,0.483,0.303,4,-10.058,1,0.0429,0.69400,0.000000,0.1150,0.139,133.406,https://api.spotify.com/v1/audio-analysis/53QF...,240166
3,0IktbUcnAGrvD03AWnz3Q8,https://p.scdn.co/mp3-preview/a89bf59628fb81ad...,acoustic,0.625,0.414,0,-8.700,1,0.0369,0.29400,0.000000,0.1510,0.669,130.088,https://api.spotify.com/v1/audio-analysis/0Ikt...,189613
4,6Uy6K3KdmUdAfelUp0SeXn,https://p.scdn.co/mp3-preview/43d2cef16aa11c7d...,acoustic,0.551,0.836,10,-3.838,0,0.0524,0.00502,0.000021,0.0425,0.758,185.063,https://api.spotify.com/v1/audio-analysis/6Uy6...,220573


In [5]:
parent_dir = "E:/music_data/"
list_file_name = []
for i in range(len(df)):
    
    genre = str(df.iloc[i].genre)
    path = os.path.join(parent_dir,genre)
    
    file_name = genre + "_" + str((i%250)+1) + ".wav"
    
    list_file_name.append(path + "/" + file_name)

In [6]:
def download_url(args):

    url, fn = args[0], args[1]
    
    try:
        path = fn.rsplit("/",1)[0]
        file_name = fn.rsplit("/",1)[1]
    except Exception as e:
        print('Exception in download_url():', e)
        
    if not os.path.exists(path):
        os.mkdir(path)
        
    if os.path.isfile(path + "/" + file_name):
        return
    
    urllib.request.urlretrieve(url, path + "/" + file_name)

In [7]:
def download_parallel(args):
    cpus = cpu_count()
    results = ThreadPool(cpus - 1).imap_unordered(download_url, args)


In [13]:
zip_music = zip(df.iloc[0:].preview_url,list_file_name)
download_parallel(zip_music)